In [32]:
import pandas as pd
from bs4 import BeautifulSoup
import glob

In [33]:
filepaths = glob.glob('data/*.htm*')

In [60]:
ten_ks = {}

for filepath in filepaths:
    with open(filepath, 'r') as f:
        content = f.read()
        soup = BeautifulSoup(content, 'html5lib')

        # Kill all script and style elements
        for script in soup(['script', 'style']):
            script.extract()

        # Get text    
        text = soup.get_text()

        # Break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.splitlines())

        # Break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))

        # Drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)

        # Append to dictionary
        ten_ks[filepath] = text